In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from helper.constantes import *
import numpy as np
import pandas as pd

In [45]:
df_spell = pd.read_json(hpcollection_folder+"spells.json")
df_spell.head(5)

,id,name,other_name,pronunciation,spell_type,description,mention,etymology,note
0,1,Aberto,None,Ah-bare-toh,Charm,None,None,None,None
1,2,Accio,Summoning Charm,"Various, including: AK-ee-oh or AK-see-oh , AK...",Charm,Summons an object towards the caster. It is ab...,Harry Potter summoned his broom to complete th...,"The Latin word accio means ""I call"" or ""I summ...",None
2,3,Age Line,None,None,Charm,Prevents people above or below a certain age f...,None,None,None
3,4,Aguamenti,Water-Making Spell,AH-gwah-MEN-tee,"Charm, Conjuration","Produces a clean, drinkable jet of water from ...",Used by Fleur Delacour in 1994 to extinguish h...,None,None
4,5,Alarte Ascendare,None,a-LAR-tay a-SEN-der-ay,Charm,Shoots the target high into the air.,Used by Gilderoy Lockhart in 1992 to send a sn...,None,None


In [53]:
occ = {}
spells = set()
spell_type = {}
for i, v in df_spell.iterrows():
    name = v['name'].lower()
    occ[name]= 0
    spells.add(name)
    spell_type[name] = v['spell_type']
    
spells.remove("pack")

In [54]:
all_lines = []
cur_line = 1
for i in range(1,8):
    # Read the full book line by line in the a list
    with open(data_folder+hpbooks_folder+f"hp{i}.txt") as f:
        lines = f.readlines()
    cur_str = ""
    for line in lines:
        if line == "\n":
            all_lines.append((cur_line, i, cur_str.replace("  ", " ").lower()))
            cur_str = ""
            cur_line += 1
        else:
            cur_str = (cur_str + " " if cur_str != "" else "") + line.replace("\”", "'").replace("“","'").replace("”","'").replace("\n","").strip()

In [58]:
occurences = []
for lineCount, book_id, line in all_lines:
    for sp in spells: 
        if sp in line:
            occurences.append((lineCount, book_id, sp,spell_type[sp], line))
            occ[sp] = occ[sp] +1

In [59]:
occurences

[(695,
  1,
  'mimblewimble',
  'Curse',
  "uncle vernon, who had gone very pale, whispered something that sounded like 'mimblewimble.' hagrid stared wildly at harry."),
 (2491,
  1,
  'switching spell',
  'Transfiguration',
  "'don’t you care about gryffindor, do you only care about yourselves, / don’t want slytherin to win the house cup, and you’ll lose all the points i got from professor mcgonagall for knowing about switching spells.'"),
 (2576,
  1,
  'alohomora',
  'Charm',
  "'oh, move over,' hermione snarled. she grabbed harry’s wand, tapped the lock, and whispered, 'alohomora'"),
 (2754,
  1,
  'wingardium leviosa',
  'Charm',
  "'wingardium leviosal' he shouted, waving his long arms like a windmill."),
 (2757,
  1,
  'wingardium leviosa',
  'Charm',
  "hermione rolled up the sleeves of her gown, flicked her wand, and said, 'wingardium leviosav’"),
 (2775,
  1,
  'purple firecrackers',
  'Charm',
  'there was an uproar. it took several purple firecrackers exploding from the end

In [60]:
import csv

# note: If you use 'b' for the mode, you will get a TypeError
# under Python3. You can just use 'w' for Python 3

with open(cleaned_folder+ 'spellByBooke.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['lineNb','bookId','spell','spellType', 'sentence'])
    for row in occurences:
        csv_out.writerow(row)